In [ ]:
;挂号可用号源
;加号 过时段可用


In [ ]:
/// @docs:	获取医生排班开始时间
/// @debug:	w ##class(web.DHCDocMain).GetScheduleStartTime(224,+$h)
/// @param:	
/// 			DoctorId:	@table CT_CareProv
/// 			Date:		%Date
/// @output:	上午开始时间^下午开始时间
ClassMethod GetScheduleStartTime(DoctorId As %String, date As %String) As %String [ Language = objectscript ]
{
	q:date="" ""
	q:DoctorId="" ""
	;s Loc = $p(^CTPCP(DoctorId,3),"^",3)
	;s ResRowId = $o(^RB("RES",0,"CTPCP",Loc,DoctorId,0))
	;^RBAS(0,"Date",{AS_Date},{RB_Resource.RES_RowId},{AS_ChildSub})
	;出诊时段
	&sql(
		SELECT RES_RowId1 into :ResRowId from SQLUser.RB_Resource 
		where RES_CTPCP_DR = :DoctorId
			and RES_CTLOC_DR->ctloc_desc like "%门诊%"
	)
	b ;;;; res
	q:$g(ResRowId)="" ""
	s time1 = "" , time2 = ""
	s ASChildSub = 0
	for {
		s ASChildSub = $o(^RBAS(0,"Date",date,ResRowId,ASChildSub))
		q:ASChildSub=""
		b ;;;
		Set SessionStartTime=$p(^RBAS(ResRowId,ASChildSub),"^",4)
		;s SessionStartTime=..%ZT(SessionStartTime,1)
		set SessionTimeID=$P($G(^RBAS(ResRowId,ASChildSub,"DHC")),"^",17)
		if SessionTimeID'="" s SessionTimeName=$P($G(^DHCTimeRange(SessionTimeID)),"^",2)
		if $g(SessionTimeName) = "上午" s time1 = SessionStartTime
		if $g(SessionTimeName) = "下午" s time2 = SessionStartTime
	}
	
	;^RB("RES",0,"CTPCP",{RES_CTPCP_DR},{RES_CTLOC_DR},{RES_RowId})
	;^RB("RES",0,"CTLOC",{RES_CTLOC_DR},{RES_RowId})
	q time1 _ "^" _ time2
}

/// @docs:	获取指定时间排班数量
/// @debug:	w ##class(web.DHCDocMain).GetScheduleCount(10,+$h,+$h)
/// @param:
/// 		Loc		@table:	CT_Loc
/// 		Start	%Date
/// 		End		%Date
/// @output:	count
ClassMethod GetScheduleCount(Loc As %String, Start As %String, End As %String) As %String [ Language = objectscript ]
{
	q:Loc="" ""
	q:Start="" ""
	q:End="" ""
	s count = 0
	for date=Start:1:End{
		s ResRowId = 0
		for {
			s ResRowId = $o(^RBAS(0,"Date",date,ResRowId))
			q:ResRowId=""
			Set DepID=$p($g(^RB("RES",ResRowId)),"^",1)
			continue:DepID'=Loc
			s ASChildSub = 0
			for {
				s ASChildSub = $o(^RBAS(0,"Date",date,ResRowId,ASChildSub))
				q:ASChildSub=""
				;状态
				Set StatusDr=$P($G(^RBAS(ResRowId,ASChildSub,"DHC")),"^",10)
				i StatusDr'="" {
					s Status=$P($G(^DHCRBCASStatus(StatusDr)),"^",1)
					s StatusDesc=$P($G(^DHCRBCASStatus(StatusDr)),"^",2)
					continue:(StatusDesc'="正常")&&((StatusDesc'="替诊"))
					s ASIrregularFlag=$p($g(^RBAS(ResRowId,ASChildSub)),"^",13)
					;默认是不找不规则出诊记录的
					continue:ASIrregularFlag="A"
					s count = count +  1
				}	
			}
		}
	}
	;^RB("RES",0,"CTPCP",{RES_CTPCP_DR},{RES_CTLOC_DR},{RES_RowId})
	;^RB("RES",0,"CTLOC",{RES_CTLOC_DR},{RES_RowId})
	q count
}

In [ ]:
pageCode:"dhcrbresourceuse.hui.csp",
pageName:"护士加号权限管理",
^User.DHCUserGroupD